In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import geopy 

In [2]:
cboList = pd.read_csv("data_cleaning/CV_CITY_CBO_KC_2022feb27_long.csv")
cboList_clean = cboList.drop(['cbo_string','cbo_count'], axis=1)
cboList_clean = cboList_clean.iloc[:,1:]
cboList_clean

,City,cbo_name
0,Anaheim,Access California Services
1,Brea,CAP OC
2,Buena Park,CHIRLA
3,Costa Mesa,AASCSC
4,Cypress,Orange County United Way
...,...,...
140,Santa Ana,Inc
141,Orange,VietRISE
142,Santa Ana,The Cambodian Family
143,Orange,WAVE


In [3]:
cboList_grpd = cboList_clean.groupby(['City'])['cbo_name'].count().reset_index()
cboList_grpd

,City,cbo_name
0,Anaheim,16
1,Brea,2
2,Buena Park,6
3,Costa Mesa,5
4,Cypress,2
5,Fountain Valley,5
6,Fullerton,8
7,Garden Grove,12
8,Huntington Beach,4
9,Irvine,9


In [4]:
cities = gpd.read_file("data_cleaning/City_Boundaries.geojson")

In [5]:
#make join columns match
cboList_grpd = cboList_grpd.rename(columns={'City':'CITY'})

#make join values match
cboList_grpd['CITY'] = cboList_grpd['CITY'].str.capitalize()
cities['CITY'] = cities['CITY'].str.capitalize()

#drop columns
cities = cities.drop(['OBJECTID','JURISDICTI','Acres','Area_SqMi','OCSurveyDBOCityBoundariesArea'], axis=1)

In [6]:
cityCBOs = cities.merge(cboList_grpd, on='CITY', how='left').fillna(0)
cityCBOs

,CITY,geometry,cbo_name
0,Unincorporated,"POLYGON ((-117.84990 33.84536, -117.85071 33.8...",0.0
1,Santa ana,"POLYGON ((-117.83141 33.77296, -117.83250 33.7...",22.0
2,Fullerton,"POLYGON ((-117.92435 33.92440, -117.92435 33.9...",8.0
3,Anaheim,"POLYGON ((-117.68346 33.87068, -117.68393 33.8...",16.0
4,Laguna niguel,"POLYGON ((-117.70083 33.56934, -117.70329 33.5...",1.0
...,...,...,...
80,San juan capistrano,"POLYGON ((-117.67414 33.54314, -117.67444 33.5...",1.0
81,Irvine,"POLYGON ((-117.67804 33.68503, -117.68089 33.6...",9.0
82,Unincorporated,"POLYGON ((-117.67761 33.86283, -117.67805 33.8...",0.0
83,Orange,"POLYGON ((-117.83173 33.84234, -117.83211 33.8...",22.0


In [7]:
cityCBOs = cityCBOs.loc[cityCBOs['CITY']!='Unincorporated']
cityCBOs = cityCBOs.drop_duplicates(subset=['CITY'], keep='first')
cityCBOs = cityCBOs.sort_values('CITY', ascending=True)
cityCBOs

,CITY,geometry,cbo_name
42,Aliso viejo,"POLYGON ((-117.71643 33.59765, -117.71682 33.6...",0.0
3,Anaheim,"POLYGON ((-117.68346 33.87068, -117.68393 33.8...",16.0
7,Brea,"POLYGON ((-117.91191 33.94608, -117.91183 33.9...",2.0
24,Buena park,"POLYGON ((-117.98535 33.89552, -117.98968 33.8...",6.0
19,Costa mesa,"POLYGON ((-117.92944 33.70201, -117.92953 33.7...",5.0
29,Cypress,"POLYGON ((-118.05877 33.84620, -118.05906 33.8...",2.0
76,Dana point,"POLYGON ((-117.68678 33.48710, -117.68687 33.4...",0.0
78,Fountain valley,"POLYGON ((-117.91535 33.72883, -117.91520 33.7...",5.0
2,Fullerton,"POLYGON ((-117.92435 33.92440, -117.92435 33.9...",8.0
84,Garden grove,"POLYGON ((-117.97194 33.80569, -117.97340 33.8...",12.0


In [15]:
cityCBOs_df = pd.DataFrame(cityCBOs)

In [20]:
cityCBOs_df.to_json(r'C:\Users\kchan\Desktop\Spring2022\JavaScript\week07\exercise\data\data_cleaning\cityCBOs.json', default_handler=str)